In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#!pip install -r requirment.txt

In [ ]:
import os

from langchain.document_loaders import UnstructuredFileLoader
from langchain_text_splitters import CharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_chroma import Chroma
from langchain_groq import ChatGroq
from langchain.chains import RetrievalQA

In [ ]:
os.environ["GROQ_API_KEY"] = "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"

In [ ]:
# Fetch the PDF from the URL
import requests
url = "https://dspmuranchi.ac.in/pdf/Blog/Python%20Built-In%20Functions.pdf"
response = requests.get(url)

In [ ]:
# Save the PDF to a local file
with open("python_inbuildfunction.pdf", "wb") as f:
    f.write(response.content)

In [ ]:
# laoding the document
loader = UnstructuredFileLoader("python_inbuildfunction.pdf")

In [ ]:
documents = loader.load()
documents

In [ ]:
text_splitter = CharacterTextSplitter(
    chunk_size=2000,
    chunk_overlap=400
)

In [ ]:
texts = text_splitter.split_documents(documents)

In [ ]:
type(texts)

In [ ]:
len(texts)

In [ ]:
texts[4]

In [ ]:
embeddings = HuggingFaceEmbeddings()

In [ ]:
persist_directory = "vector_db"

In [ ]:
vectordb = Chroma.from_documents(
    documents=texts,
    embedding=embeddings,
    persist_directory=persist_directory
)

In [ ]:
# retriever
retriever = vectordb.as_retriever()

In [ ]:
# llm from groq
llm = ChatGroq(
    model="llama-3.1-70b-versatile",
    temperature=0
)

In [ ]:
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True
)

In [ ]:
# invoke the qa chain and get a response for user query
query = "what are the function from this pdf "
response = qa_chain.invoke({"query": query})

In [ ]:
print(response)

In [ ]:
print(response["result"])

In [ ]:
print(response["source_documents"][0].metadata["source"])

In [ ]:
# invoke the qa chain and get a response for user query
query = "Give me summary of all function from this pdf?"
response = qa_chain.invoke({"query": query})
print(response["result"])
print("*"*30)
print("Source Document:", response["source_documents"][0].metadata["source"])